--Imports--

In [ ]:
import os
from typing import TypedDict
from typing import List
from typing import Optional
from langchain.Schema import Document
from langgraph.graph import StateGraph
from langgraph.graph import END
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.dog_search.tool import DuckDuckGoSearchRun

--- Load enviroment variables and API keys---

In [ ]:
from dotenv import load_dotenv
load_dotenv()

--- Load $ Index medical PDF for RAG --- 

In [ ]:
loader = PyPDFLoader('../data/medical_book.pdf')
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 512 , 
    chunk_overlap = 128 , 
    seperators=["\n\n", ". " , "\n", " "]
)

In [ ]:
doc_splits = text_splitter.split_documents(docs)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectorstore = Chroma.from_documents(
    documents=doc_splits , 
    embeddings=embeddings , 
    persist_directory="../medical_db/" , 
    collection_metadata={"hnsw:space" : "cosine"}
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

--- Initialize LLM ---

In [ ]:
llm = ChatGroq(
    model_name="openai/gpt-oss-120b" , 
    temperature = 0.3 , 
    max_tokens = 2048
)

--- Initialize external tools ---

In [ ]:
wiki = WikipediaAPIWrapper(
    api_wrapper = WikipediaAPIWrapper(
        top_k_results = 2 , 
        doc_content_chars_max=2000 , 
        load_all_available_meta=True
    )
)

In [ ]:
duckduckgo_search = DuckDuckGoSearchRun()

--- Define AgentState TypedDict with Success/failure flags ---

In [ ]:
class AgentState(TypedDict) : 
    question: str
    documents: List[Document]
    generation: str
    source: str
    search_query: Optional[str]
    conversation_history: List[str]
    llm_attempted: bool
    llm_success: bool
    rag_attempted: bool
    rag_success: bool
    wiki_attempted: bool
    wiki_success: bool
    ddg_attempted: bool
    ddg_success: bool
    current_tool: Optional[str]
    retry_count: int

--- Memory agent (maintain short-term conversation buffer)

In [ ]:
def MemoryAgent(state : AgentState) -> AgentState :
    history = state.get("conversation_history" , [])
    if len(history) > 20 :
        history = history[-20:]
    state["conversation_history"] = history
    return state

--- LLM Agent : first attempt to answer ---

In [ ]:
def LLMAgent(state : AgentState) -> AgentState :

    try : 
        ctx = "\n".join(state.get("conversation_history" , [] )[-10:])

    except Exception : 
        state["llm_success"] = False

    state["llm_attempted"] = True
    return state

--- LLM Agent : first attempt to answer ---

In [ ]:
def LLMAgent(state : AgentState) -> AgentState : 

    try : 
        ctx="\n".join(state.get("conversation_history" , [])[-10 : ])
        prompt = f"""You are a compassionate and knowledgeable medical AI assistant and doctor helping a patient. Your conversational skill should be a professional consultant with a human touch.

        Patient's History : 
        {ctx}
        Patient's Question : 
        {state["question"]}


        Respond like an experienced doctor in 2–3 sentences. Be clear, professional, and confident. Do not mention sources or uncertainty."""

        response = llm.invoke(prompt)
        answer = response.content.strip()

        if answer : 
            state["generation"] = answer
            state["llm_success"] = True
        else :
            state["llm_success"] = False

    except Exception : 
        state["llm_success"] = False

    state["llm_attempted"] = True
    return state

--- Planner Agent : initial tool decision based on query keywords ---

In [ ]:
def PlannerAgent(state : AgentState) -> AgentState :
    question = state["question"]
    medical_keywords = ["pain", "fever", "treatment", "symptom", "diagnosis", "cancer", "disease", "virus", "bacteria", "infection"]

    if any(word in question for word in medical_keywords) :
        state["current_tool"] = "llm"
    else : 
        state["current_tool"] = "llm"

    state["retry_count"] = 0  
    return state


--- Retriever Agent (RAG) from PDF vectorstore

In [ ]:
def RetrieverAgent(state : AgentState) -> AgentState : 
    query = state["question"]
    context = "\n".join(state.get("conversation_history" , [])[-6:])

    combined_query = f"Context : {context}\nQuestion : {query}"

    try : 
        docs == retriever.invoke(combined_query)

        if docs and len(docs) > 0 : 
            state["documents"] = docs
            state["rag_success"] =  True
            state["conversation_history"].append("AI : Retrieved documents from medical PDF database.")

        else : 
            state["documents"] = []
            state["rag_success"] = False

    except Exception : 
        state["documents"] = []
        state["rag_success"] = False

    state["rag_attempted"] = True
    return state

-- WikiPedia Agent Fallback ----

In [ ]:
def WikipediaAgent(state : AgentState) -> AgentState :
    try : 
        content = wiki.run(state["question"])

        if content : 
            state["documents"] = [Document(page_content=content)]
            state["wiki_success"] = True
            state["conversation_history"].append("AI : Retrieved Information from Wikipedia")

        else : 
            state["documents"] = [] 
            state["wiki_success"] = False

    except Exception :
        state["documents"] = []
        state["wiki_success"] = False
    
    state["wiki_attempted"] = True
    return state

NameError: name 'AgentState' is not defined

--- DuckDuckGo Agent fallback --- -

In [ ]:
def DuckDuckGoAgent(state : AgentState) -> AgentState : 
    try : 
        content = duckduckgo_search.run(state["question"])
        if content : 
            state["documents"] = [Document(page_content = content)]
            state["ddg_success"] = True
            state["conversation_history"].append("AI : Retrieved information from DuckDuckGo")
        else : 
            state["documents"] = []
            state["ddg_success"] = False

    except Exception : 
        state["documents"] = []
        state["ddg_success"] = False
    
    state["ddg_attempted"] = True
    return state

--- executor Agent - Generate final answer using LLM with retrieved docs or fallback to knowledge ---

In [ ]:
def ExecutorAgent(state : AgentState) -> AgentState : 
    context = state.get("conversation_history" , [])
    question = state["question"]

    if state.get("documents") and len(state["documents"]) > 0 :
        content = "\n".join([doc.page_content for doc in state["documents"]])
        prompt = f"""You are a kind, highly experienced professional medical doctor speaking directly with a patient. Be clear, supportive and concise like human response.
        Conversation Context:
        {"".join(context[-6:])}

        Patient's Question:
        {question}

        Relevant Medical Information:
        {content}

        Guidelines:
        - Answer in 2-3 sentences.
        - Do not mention sources.
        - Speak like a caring human doctor."""

        response = llm.invoke(prompt)
        answer = response.content.strip()
        state["generation"] = answer
        state["source"] = "retrieved_docs"
        state["conversation_history"].append(f"Doctor: {answer}")
        return state
    
    if state.get("locals" , False) and state.get("generation") : 
        state["conversation_history"].append(f"Doctor : {state['generation']}")
        state["source"] = "llm_knowledge"
        return state
    
    state["generation"] = "I could not find enough information to answer" 
    state["source"] = "none"
    state["conversation_history"].append(state["generation"])
    return state

                


--- Explanation Agent (append explanation , confidence , traceability) ----


In [ ]:
def ExplanationAgent(state : AgentState) -> AgentState : 
    explanation = "This response is generated using a combination of medical literature and AI reasoning."
    state["conversation_history"].append(f"AI Explanation : {explanation}")

    return state

-- Build Langgraph workflows ---

In [ ]:
workflow = StateGraph(AgentState)

---- Add all agent nodes ----

In [ ]:
workflow.add_node("memory" , MemoryAgent)
workflow.add_node("planner" , PlannerAgent)
workflow.add_node("llm_agent" , LLMAgent)
workflow.add_node("retriever" , RetrieverAgent)
workflow.add_node("wikipedia" , WikipediaAgent)
workflow.add_node("duckduckgo" , DuckDuckGoAgent)
workflow.add_node("executor" , ExecutorAgent)
workflow.add_node("explanation" , ExplanationAgent)


--- Set Entry Point ---

In [ ]:
workflow.set_entry_point("memory")

--- Edges and conditional routing functions for fallback chain ---

In [ ]:
workflow.add_edge("memory" , "planner")
workflow.add_edge("planner" , "llm_agent")

#after LLM Agent
def route_after_llm(state  : AgentState) :
    if state.get("llm_success" , False) : 
        return "executor"
    else : 
        return retriever
    
workflow.add_conditional_edges("llm_agent" , route_after_llm , {"executor" : "executor" , "retriever" : "retriever"})

#after retriever agent
def route_after_rag(state : AgentState) : 
    if state.get("rag_success" , False) : 
        return "executor"
    else : 
        return "wikipedia"
    
workflow.add_conditional_edges("retriever" , route_after_rag , {"executor" : "executor" , "wikipedia" : "wikipedia"})


#After wikipedia agent
def route_after_wiki(state : AgentState) : 
    if state.get("wiki_success" , False) : 
        return "executor"
    else : 
        return "duckduckgo"
    
workflow.add_conditional_edges("wikipedia" , route_after_wiki , {"executor" : "executor" , "duckduckgo" : "duckduckgo"})

#After Duckcuckgo agent
def route_after_ddg(state : AgentState) -> AgentState : 
    return "executor"

workflow.add_conditional_edges("duckduckgo" , route_after_ddg , {"executor" : "executor"})

#Executor to explanation then end
workflow.add_edge("executor" , 'explanation')
workflow.add_edge('explanation' , END)

--- Compile the workflow ----

In [ ]:
app = workflow.compile()

--- Initialize conversation state ---

In [ ]:
conversation_state : AgentState = {
    "question" : "" , 
    "documents" : [] , 
    "generation" : "" , 
    "source" : "" , 
    "search_query" : None , 
    "conversation_history" : [] , 
    "llm_attempted" : False , 
    "llm_success" : False , 
    "rag_attempted" : False  , 
    "rag_success" : False , 
    "wiki_attempted" : False , 
    "wiki_success" : False , 
    "ddg_attempted" : False , 
    "ddg_success" : False , 
    "current_tool" : None , 
    "retry_count" : 0
}

--- Main INteraction loop ---

In [ ]:
print("=== Medical AI Assistant (Type exit to quit) ===")
while True : 
    query = input("\nAsk your medical question : ").strip()

    if query.lower() == "exit" : 
        conversation_state = {
            "question" : "" , 
            "documents" : [] , 
            "generation" : "" , 
            "source" : "" , 
            "search_query" : None , 
            "conversation_history" : [] , 
            "llm_attempted" : False , 
            "llm_success" : False , 
            "rag_attempted" : False , 
            "rag_success" : False , 
            "wiki_attempted" : False , 
            "wiki_success" : False , 
            "ddg_attempted" : False , 
            "ddg_success" : False , 
            "current_tool" : None , 
            "retry_count" : 0 ,
        }
        print("\n=== Consultation Ended. Conversation history cleared===")
        break  

    #update conversation state with new question
    conversation_state.update({
        "question" : query , 
        "documents" : [] , 
        "generation" : "" , 
        "source" : "" , 
        "search_query" : None , 
        "llm_attempted" : False , 
        "llm_success" : False , 
        "rag_attempted" : False , 
        "rag_success" : False , 
        "wiki_attempted" : False , 
        "wiki_success"  : False , 
        "ddg_attempted" : False , 
        "ddg_success" : False , 
        "current_tool" : None , 
        "retry_count" : 0
    })

    #Run the langgraph workflow with current state
    result = app.invoke(conversation_state)
    conversation_state.update(result)

    #print AI response
    if result.get("generation") : 
        print(f"\n[Doctor AI] {result['generation']}")
    else :
        print("\n[Doctor AI] Sorry , I couldnot generate an answer")

    print("\n" + "-" * 60)